## Group #3


1. Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

In [ ]:
# pip install rasterio
# %pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
# !pip install ipywidgets
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# pip install rasterstats
# %pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
# %pip install git+https://github.com/jgrss/geowombat  -q

In [24]:

import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import mapping
import rasterio
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import os
from rasterio.mask import mask
from rasterstats import zonal_stats
import pandas as pd
#import geowombat as gw ## Chequear gw en collab



In [27]:
import geopandas as gpd

# Ruta al shapefile
shapefile_path = r'../../_data/INEI_LIMITE_DEPARTAMENTAL'

# Leer el shapefile usando Geopandas
departmentos = gpd.read_file(shapefile_path)
departmentos = departments.to_crs('esri:54009')

print(departmentos.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [28]:
# Rutas en collab

raster = ["drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif"
         ]

statistics = []

In [ ]:
# Iterar sobre cada archivo raster
for raster_path in raster:
    # estadísticas zonales
    stats = zonal_stats(departmentos, raster_path, stats="count sum", categorical=True, all_touched=True)
    # transforma DF
    stats_df = pd.DataFrame(stats)
    # Concatenar 
    df = pd.concat([departmentos.reset_index(drop=True), stats_df], axis=1)
    statistics.append(df)

In [ ]:
# Concatenar todos los DataFrames en all_stats en un único DataFrame
final_df = pd.concat(statistics, ignore_index=True)

print(final_df.head())


2. Then you are going to generate choropleth map using folium for these 15 variables.

3. Save your html in the same folder of your JN. Name your HTML as your branch. This HTML should have all these layers. Please do not forget to use Layer Control.
